In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate

np.random.seed(12345)


def initialize(input_dim, hidden_dim, output_dim, batchsize):
    W1 = np.random.randn(hidden_dim, input_dim) * 0.01
    b1 = np.zeros((hidden_dim,))
    W2 = np.random.randn(hidden_dim, hidden_dim) * 0.01
    b2 = np.zeros((hidden_dim,))
    W3 = np.random.randn(output_dim, hidden_dim) * 0.01
    b3 = np.zeros((output_dim,))

    parameters = [W1, b1, W2, b2, W3, b3]
    x = np.random.rand(input_dim, batchsize)
    y = np.random.randn(output_dim, batchsize)
    return parameters, x, y


def sigmoid(x):
    return np.divide(1.0, np.add(1.0, np.exp(-x)))


def dsigmoid(x):
    # input x is already sigmoid
    return x * (1.0 - x)


def loss(pred, y):
    return np.multiply(1 / y.shape[1],
                       np.sum(np.multiply(.5, np.sum(np.power(np.subtract(pred, y), 2), axis=0)), axis=0))


def dloss(pred, y):
    return np.multiply(1 / y.shape[1], np.subtract(pred, y))

In [12]:

class NeuralNet(object):

    def __init__(self, batch_size, input_dim=3, hidden_dim=4, output_dim=2):
        
        self.batch_size = batch_size

        # size of layers
        self.input_dim = input_dim
        self.hidden_dim_1 = hidden_dim
        self.hidden_dim_2 = hidden_dim
        self.output_dim = output_dim

        # activations
        # just take ones as they are overwritten anyway
        self.ai = np.ones((self.input_dim, self.batch_size))
        self.ah1 = np.ones((self.hidden_dim_1, self.batch_size))
        self.ah2 = np.ones((self.hidden_dim_2, self.batch_size))
        self.ao = np.ones((self.output_dim, self.batch_size))

        parameters, self.x, self.y = initialize(input_dim, hidden_dim, output_dim, batch_size)

        self.W1, self.b1, self.W2, self.b2, self.W3, self.b3 = parameters

    def forward_pass(self, x):
        # input activations
        self.ai = x

        # hidden_1 activations
        self.ah1 = sigmoid(np.add(np.dot(self.W1, self.ai), self.b1[:, np.newaxis]))

        # hidden_2 activations
        self.ah2 = sigmoid(np.add(np.dot(self.W2, self.ah1), self.b2[:, np.newaxis]))

        # output activations
        self.ao = np.add(np.dot(self.W3, self.ah2), self.b3[:, np.newaxis])

        # return outputs for display
        return np.concatenate((self.ai, self.ah1, self.ah2, self.ao))

    def backward_pass(self):

        out_error = loss(self.ao, self.y)

        print("Error: {}".format(out_error))

        # derivative of output function f(x) = x is f'(x) = 1
        out_delta = dloss(self.ao, self.y) * 1

        # calculate error for hidden_2
        hidden_2_error = np.dot(self.W3.T, out_delta)
        hidden_2_delta = np.multiply(hidden_2_error, dsigmoid(self.ah2))

        # calculate error for hidden_1
        hidden_1_error = np.dot(self.W2.T, hidden_2_delta)
        hidden_1_delta = np.multiply(hidden_1_error, dsigmoid(self.ah1))

        w1_deriv = np.dot(hidden_1_delta, self.ai.T)
        b1_deriv = np.sum(hidden_1_delta, axis=1)

        w2_deriv = np.dot(hidden_2_delta, self.ah1.T)
        b2_deriv = np.sum(hidden_2_delta, axis=1)

        w3_deriv = np.dot(out_delta, self.ah2.T)
        b3_deriv = np.sum(out_delta, axis=1)

        return [w1_deriv, b1_deriv, w2_deriv, b2_deriv, w3_deriv, b3_deriv]

    def check_gradients(self, eps=1e-4):

        # iterate through the network once to get activations
        output = self.forward_pass(self.x)

        # backprop for comparison
        gradients = self.backward_pass()

        dw1, db1, dw2, db2, dw3, db3 = gradients
        grad = np.concatenate((dw1.ravel(), db1.ravel(), dw2.ravel(), db2.ravel(),
                               dw3.ravel(), db3.ravel()))

        self.visualize(output, gradients)

        # inital weights as 1D theta array
        params = np.concatenate((self.W1.ravel(), self.b1.ravel(), self.W2.ravel(),
                                 self.b2.ravel(), self.W3.ravel(), self.b3.ravel()))

        # container
        j_plus = np.zeros((params.shape[0],))
        j_minus = np.zeros((params.shape[0],))
        grad_numeric = np.zeros((params.shape[0],))

        # Compute numeric grad
        for i in range(len(params)):
            theta_plus = np.copy(params)
            theta_plus[i] += eps
            w_plus = self.array_to_weights(theta_plus)
            j_plus[i] = loss(self.predict(self.x, w_plus), self.y)

            theta_minus = np.copy(params)
            theta_minus[i] -= eps
            w_minus = self.array_to_weights(theta_minus)
            j_minus[i] = loss(self.predict(self.x, w_minus), self.y)

            # Compute approx. gradient
            grad_numeric[i] = np.subtract(j_plus[i], j_minus[i]) / (2 * eps)

        # compute relative error as propsed in cs231n
        difference = np.divide(np.linalg.norm(np.subtract(grad, grad_numeric)),
                               max(np.linalg.norm(grad), np.linalg.norm(grad_numeric)))

        if difference < 1e-7:
            print("\033[92m" + "Backprop works perfectly! Difference = " + str(
                difference) + "\033[0m")

        elif difference < 1e-4:
            print("\033[93m" + "Backprop is usually okay! Difference = " + str(
                difference) + "\033[0m")
        else:
            print("\033[91m" + "Error in backprop (or gradient check)! Difference = " + str(
                difference) + "\033[0m")

        return difference

    def predict(self, x, weights):
        self.W1, self.b1, self.W2, self.b2, self.W3, self.b3 = weights
        return self.forward_pass(x)[-self.output_dim:]

    def array_to_weights(self, array):
        w1_idx = self.W1.ravel().shape[0]
        b1_idx = w1_idx + self.b1.ravel().shape[0]
        w2_idx = b1_idx + self.W2.ravel().shape[0]
        b2_idx = w2_idx + self.b2.ravel().shape[0]
        w3_idx = b2_idx + self.W3.ravel().shape[0]
        b3_idx = w3_idx + self.b3.ravel().shape[0]

        W1 = array[0:w1_idx].reshape(self.W1.shape)
        b1 = array[w1_idx:b1_idx].reshape(self.b1.shape)
        W2 = array[b1_idx:w2_idx].reshape(self.W2.shape)
        b2 = array[w2_idx:b2_idx].reshape(self.b2.shape)
        W3 = array[b2_idx:w3_idx].reshape(self.W3.shape)
        b3 = array[w3_idx:b3_idx].reshape(self.b3.shape)

        return [W1, b1, W2, b2, W3, b3]

    @staticmethod
    def visualize(activations, gradients):
        
        df = pd.DataFrame(activations).T
        df.columns = ["x1", "x2", "x3", "h11", "h12", "h13", "h14", "h21", "h22", "h23", "h24", "o1", "o2"]
        # round by 3 decimals to shown in one line
        print(tabulate(df, headers='keys', tablefmt='psql', floatfmt=".3f"))
        print("\n")

        dw1, db1, dw2, db2, dw3, db3 = gradients

        print("dL/dW1:\n {}\n".format(dw1))
        print("dL/db1:\n {}\n".format(db1))
        print("dL/dW2:\n {}\n".format(dw2))
        print("dL/db2:\n {}\n".format(db2))
        print("dL/dW3:\n {}\n".format(dw3))
        print("dL/db3:\n {}\n".format(db3))

In [13]:
def run(batch_size=5):
    # X = np.array([[.13, .68, .80, .57, .97],
    #               [.63, .89, .50, .35, .71],
    #               [.50, .23, .24, .79, .50]])

    nn = NeuralNet(batch_size=batch_size, input_dim=3, hidden_dim=4, output_dim=2)
    nn.check_gradients()

In [14]:
run()

Error: 0.9992378792255207
+----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+-------+
|    |    x1 |    x2 |    x3 |   h11 |   h12 |   h13 |   h14 |   h21 |   h22 |   h23 |   h24 |     o1 |    o2 |
|----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+-------|
|  0 | 0.472 | 0.826 | 0.420 | 0.501 | 0.497 | 0.499 | 0.499 | 0.498 | 0.502 | 0.501 | 0.498 | -0.020 | 0.011 |
|  1 | 0.721 | 0.190 | 0.768 | 0.504 | 0.500 | 0.500 | 0.498 | 0.498 | 0.502 | 0.501 | 0.498 | -0.020 | 0.011 |
|  2 | 0.189 | 0.020 | 0.963 | 0.504 | 0.502 | 0.501 | 0.500 | 0.498 | 0.502 | 0.501 | 0.498 | -0.020 | 0.011 |
|  3 | 0.435 | 0.703 | 0.893 | 0.503 | 0.499 | 0.500 | 0.500 | 0.498 | 0.502 | 0.501 | 0.498 | -0.020 | 0.011 |
|  4 | 0.823 | 0.347 | 0.135 | 0.501 | 0.497 | 0.498 | 0.498 | 0.498 | 0.502 | 0.501 | 0.498 | -0.020 | 0.011 |
+----+-------+-------+-------+-------+-------+-------+-------+-------+-------+